In [1]:
import configuration
from notebook_utils import notebookToPython


config = configuration.ConfigLoader('config.yml')

notebookToPython(config.projectName)
job = config.getJobConfig(config.activeJobId)

Write python file


In [2]:
from label_reader import readLabels


trainingLabels = readLabels(job)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt...


In [ ]:
fullDataPath = job.fullJoinFilePath(job.dataPath, job.trainingDataPath)

for file_name, label in trainingLabels.items():
    audioSourceFilename = job.fullJoinFilePath(fullDataPath, file_name + job.trainingDataExtension)
    
    audio, _ = librosa.load(audioSourceFilename, sr = job.sampleRate, duration = job.duration)

C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac/LA_T_1138215.flac
C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac/LA_T_1271820.flac
C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac/LA_T_1272637.flac
C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac/LA_T_1276960.flac
C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac/LA_T_1341447.flac
C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac/LA_T_1363611.flac
C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac/LA_T_1596451.flac
C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac/LA_T_1608170.flac
C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac/LA_T_1684951.flac
C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_train/flac/LA_T_169